In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read data
data = pd.read_csv("conglomerated-all.csv")
data.columns

Index(['Date', 'Client_Future Index Long', 'Client_Future Index Short',
       'Client_Option Index Call Long', 'Client_Option Index Put Long',
       'Client_Option Index Call Short', 'Client_Option Index Put Short',
       'DII_Future Index Long', 'DII_Future Index Short',
       'DII_Option Index Call Long', 'DII_Option Index Put Long',
       'DII_Option Index Call Short', 'DII_Option Index Put Short',
       'FII_Future Index Long', 'FII_Future Index Short',
       'FII_Option Index Call Long', 'FII_Option Index Put Long',
       'FII_Option Index Call Short', 'FII_Option Index Put Short',
       'Total Call Options - Open Interest BANK NIFTY INDEX',
       'Total Put Options - Open Interest BANK NIFTY INDEX',
       'Total Futures - Open Interest BANK NIFTY INDEX',
       'Price - BANK NIFTY INDEX', 'Returns - BANK NIFTY INDEX',
       'NSE VIX - India', 'is_expiry', 'Total_OI'],
      dtype='object')

In [24]:
# Add day of week column
data["DayOfWeek"] = pd.to_datetime(data["Date"]).dt.dayofweek

In [25]:
import statsmodels.api as sm

# Calculate total Client OI (sum of all client positions)
data['Client_Total_OI'] = (
    data['Client_Future Index Long'] + data['Client_Future Index Short'] +
    data['Client_Option Index Call Long'] + data['Client_Option Index Put Long'] +
    data['Client_Option Index Call Short'] + data['Client_Option Index Put Short']
)

# Y1: Retail Share
data['Retail_Share'] = (data['Client_Total_OI'] / data['Total_OI']) * 100

# Y2: Retail Short Ratio
data['Client_Short_OI'] = (
    data['Client_Future Index Short'] +
    data['Client_Option Index Call Short'] +
    data['Client_Option Index Put Short']
)
data['Retail_Short_Ratio'] = data['Client_Short_OI'] / data['Client_Total_OI']

# Y3: Retail Put Share
data['Client_Put_OI'] = data['Client_Option Index Put Long'] + data['Client_Option Index Put Short']
data['Client_Call_OI'] = data['Client_Option Index Call Long'] + data['Client_Option Index Call Short']
data['Retail_Put_Share'] = data['Client_Put_OI'] / (data['Client_Put_OI'] + data['Client_Call_OI'])

# Prepare independent variables
X = data[['DayOfWeek', 'NSE VIX - India', 'Returns - BANK NIFTY INDEX']].copy()
X = sm.add_constant(X)  # Adds the constant term (α)

regression_data = pd.concat([X, data[['Retail_Share', 'Retail_Short_Ratio', 'Retail_Put_Share']]], axis=1)
regression_data = regression_data.dropna()
# Update X and Y variables after dropping NaNs
X_clean = regression_data[['const', 'DayOfWeek', 'NSE VIX - India', 'Returns - BANK NIFTY INDEX']]

In [26]:
model1 = sm.OLS(regression_data['Retail_Share'], X_clean)
results1 = model1.fit()
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:           Retail_Share   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     28.89
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           2.67e-15
Time:                        20:53:15   Log-Likelihood:                -396.69
No. Observations:                 184   AIC:                             801.4
Df Residuals:                     180   BIC:                             814.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [27]:
model2 = sm.OLS(regression_data['Retail_Short_Ratio'], X_clean)
results2 = model2.fit()
print(results2.summary())


                            OLS Regression Results                            
Dep. Variable:     Retail_Short_Ratio   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     4.347
Date:                Wed, 12 Nov 2025   Prob (F-statistic):            0.00553
Time:                        20:53:15   Log-Likelihood:                 572.30
No. Observations:                 184   AIC:                            -1137.
Df Residuals:                     180   BIC:                            -1124.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [28]:
model3 = sm.OLS(regression_data['Retail_Put_Share'], X_clean)
results3 = model3.fit()
print(results3.summary())

# The outcome has a very low R^2 of 0.037, indicating that the independent variables explain only a small portion of the variance in Retail Put Share. 
# This indicates the extent of hedging or downside-protection orientation among retail traders is not well described by the 
# day of the week, market volatility, or recent returns. Of the independent variables, only Returns - BANK NIFTY INDEX is statistically significant (p < 0.05),

                            OLS Regression Results                            
Dep. Variable:       Retail_Put_Share   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.5113
Date:                Wed, 12 Nov 2025   Prob (F-statistic):              0.675
Time:                        20:53:15   Log-Likelihood:                 301.75
No. Observations:                 184   AIC:                            -595.5
Df Residuals:                     180   BIC:                            -582.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [29]:
# To "regroup" and assess the relevant independent variables to include for statistical viability, we can perform correlation analysis and 
# VIF (Variance Inflation Factor) checks on the independent variables.
# This will help identify multicollinearity issues and determine which variables contribute meaningfully to the model.

# 1. Check correlations with dependent variable
correlations = data[['Retail_Share', 'DayOfWeek', 'NSE VIX - India', 
                     'Returns - BANK NIFTY INDEX', 'is_expiry']].corr()
print(correlations['Retail_Share'].sort_values(ascending=False))

# 2. Check multicollinearity among independent variables
from statsmodels.stats.outliers_influence import variance_inflation_factor

regression_cols = ['Date', 'DayOfWeek', 'NSE VIX - India', 'Returns - BANK NIFTY INDEX',
                   'Retail_Share', 'Retail_Short_Ratio', 'Retail_Put_Share', 'is_expiry']

clean_data = data[regression_cols].copy()

# Replace infinite values with NaN
clean_data = clean_data.replace([np.inf, -np.inf], np.nan)

# Drop rows with any missing values
clean_data = clean_data.dropna()

print(f"Original dataset: {len(data)} rows")
print(f"Clean dataset: {len(clean_data)} rows")
print(f"Rows dropped: {len(data) - len(clean_data)}")

# Now use clean_data for VIF and regressions
X_vars = clean_data[['DayOfWeek', 'NSE VIX - India', 'Returns - BANK NIFTY INDEX']]

vif_data = pd.DataFrame()
vif_data["Variable"] = X_vars.columns
vif_data["VIF"] = [variance_inflation_factor(X_vars.values, i) 
                   for i in range(len(X_vars.columns))]
print("\nVIF Results:")
print(vif_data)


Retail_Share                  1.000000
is_expiry                    -0.004555
DayOfWeek                    -0.048970
Returns - BANK NIFTY INDEX   -0.049504
NSE VIX - India              -0.561025
Name: Retail_Share, dtype: float64
Original dataset: 184 rows
Clean dataset: 184 rows
Rows dropped: 0

VIF Results:
                     Variable       VIF
0                   DayOfWeek  2.708619
1             NSE VIX - India  2.701170
2  Returns - BANK NIFTY INDEX  1.014459


In [30]:
# Based on the prior results - no independent variables exhibit high multicollinearity (VIF < 5)
# In terms of correlation with the dependent variable (Retail_Share), NSE VIX - India shows the highest correlation, with some from DayOfWeek as well.

# In light of these findings, I will test to see if other independent variables or transformations (ex., interaction terms, polynomial terms)
# might better capture the dynamics influencing retail trading behavior in the Indian stock market.

# FII Net Position (smart money indicator)
clean_data['FII_Net_Position'] = (
    data['FII_Future Index Long'] - data['FII_Future Index Short'] +
    data['FII_Option Index Call Long'] - data['FII_Option Index Call Short'] +
    data['FII_Option Index Put Short'] - data['FII_Option Index Put Long']
)

# FII Share of total OI (institutional dominance)
clean_data['FII_Share'] = (
    (data['FII_Future Index Long'] + data['FII_Future Index Short'] +
     data['FII_Option Index Call Long'] + data['FII_Option Index Put Long'] +
     data['FII_Option Index Call Short'] + data['FII_Option Index Put Short']) /
    data['Total_OI']
) * 100


# Lagged returns (momentum/reversal effects)
clean_data['Returns_Lag1'] = data['Returns - BANK NIFTY INDEX'].shift(1)
clean_data['Returns_Lag2'] = data['Returns - BANK NIFTY INDEX'].shift(2)
clean_data['Returns_Lag5'] = data['Returns - BANK NIFTY INDEX'].shift(5)  # weekly

# Cumulative returns (recent performance)
clean_data['Returns_5day'] = data['Returns - BANK NIFTY INDEX'].rolling(5).sum()
clean_data['Returns_20day'] = data['Returns - BANK NIFTY INDEX'].rolling(20).sum()

# Lagged volatility
clean_data['VIX_Lag1'] = data['NSE VIX - India'].shift(1)
clean_data['VIX_Lag5'] = data['NSE VIX - India'].shift(5)

# Change in VIX (volatility shock indicator)
clean_data['VIX_Change'] = data['NSE VIX - India'].diff()
clean_data['VIX_Spike'] = (clean_data['VIX_Change'] > clean_data['VIX_Change'].quantile(0.75)).astype(int)

# High vs low volatility regime
clean_data['High_VIX_Regime'] = (data['NSE VIX - India'] > data['NSE VIX - India'].median()).astype(int)

# Bull vs bear market
clean_data['Bull_Market'] = (clean_data['Returns_20day'] > 0).astype(int)

# Winning vs losing days
clean_data['Positive_Return'] = (data['Returns - BANK NIFTY INDEX'] > 0).astype(int)

# Market stress indicator (high VIX + negative returns)
clean_data['Market_Stress'] = ((data['NSE VIX - India'] > data['NSE VIX - India'].quantile(0.75)) & 
                                (data['Returns - BANK NIFTY INDEX'] < 0)).astype(int)

# Historical volatility (standard deviation of returns)
clean_data['Realized_Vol_5d'] = data['Returns - BANK NIFTY INDEX'].rolling(5).std() * np.sqrt(252)
clean_data['Realized_Vol_20d'] = data['Returns - BANK NIFTY INDEX'].rolling(20).std() * np.sqrt(252)

# Range-based volatility
clean_data['Daily_Range'] = ((data['Price - BANK NIFTY INDEX'].rolling(2).max() - 
                              data['Price - BANK NIFTY INDEX'].rolling(2).min()) / 
                             data['Price - BANK NIFTY INDEX']) * 100

# Days until next expiry (if you can calculate this)
# Bank Nifty options expire weekly on Wednesdays
clean_data['Days_To_Expiry'] = clean_data['DayOfWeek'].apply(
    lambda x: (2 - x) % 7 if x <= 2 else (9 - x)  # 0=Mon, 2=Wed expiry
)

# Week of expiry indicator
clean_data['Expiry_Week'] = (clean_data['Days_To_Expiry'] <= 2).astype(int)

# Model 1: Add institutional positioning
X1 = clean_data[['DayOfWeek', 'NSE VIX - India', 'Returns - BANK NIFTY INDEX', 
                 'FII_Net_Position']]

# Model 2: Add lagged effects
X2 = clean_data[['DayOfWeek', 'NSE VIX - India', 'Returns_Lag1', 'VIX_Lag1']]

# Model 3: Add market stress
X3 = clean_data[['DayOfWeek', 'NSE VIX - India', 'Returns - BANK NIFTY INDEX',
                 'Market_Stress', 'VIX_Spike']]

# Model 4: Add expiry dynamics
X4 = clean_data[['DayOfWeek', 'NSE VIX - India', 'Returns - BANK NIFTY INDEX',
                 'Days_To_Expiry', 'is_expiry']]

# Model 5: Comprehensive model
X5 = clean_data[['DayOfWeek', 'NSE VIX - India', 'Returns_Lag1', 
                 'FII_Net_Position', 'Market_Stress', 'Days_To_Expiry']]

In [31]:
X1 = sm.add_constant(X1)
model4 = sm.OLS(regression_data['Retail_Share'], X1)
results4 = model4.fit()
print(results4.summary())

                            OLS Regression Results                            
Dep. Variable:           Retail_Share   R-squared:                       0.362
Model:                            OLS   Adj. R-squared:                  0.348
Method:                 Least Squares   F-statistic:                     25.44
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           1.06e-16
Time:                        20:53:15   Log-Likelihood:                -391.44
No. Observations:                 184   AIC:                             792.9
Df Residuals:                     179   BIC:                             809.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [32]:
# We still extremely high multicollinearity with these added variables, need to STANDARDIZE 

from sklearn.preprocessing import StandardScaler

# Standardize all continuous variables to mean=0, std=1
scaler = StandardScaler()
vars_to_scale = ['NSE VIX - India', 'Returns - BANK NIFTY INDEX', 'FII_Net_Position']

clean_data_scaled = clean_data.copy()
clean_data_scaled[vars_to_scale] = scaler.fit_transform(clean_data[vars_to_scale])

# Run regression with scaled variables
X1_scaled = clean_data_scaled[['DayOfWeek', 'NSE VIX - India', 
                                'Returns - BANK NIFTY INDEX', 'FII_Net_Position']]
X1_scaled = sm.add_constant(X1_scaled)

model_scaled = sm.OLS(clean_data_scaled['Retail_Share'], X1_scaled).fit()
print(model_scaled.summary())

                            OLS Regression Results                            
Dep. Variable:           Retail_Share   R-squared:                       0.362
Model:                            OLS   Adj. R-squared:                  0.348
Method:                 Least Squares   F-statistic:                     25.44
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           1.06e-16
Time:                        20:53:15   Log-Likelihood:                -391.44
No. Observations:                 184   AIC:                             792.9
Df Residuals:                     179   BIC:                             809.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [34]:
model_scaled2 = sm.OLS(clean_data_scaled['Retail_Short_Ratio'], X1_scaled).fit()
print(model_scaled2.summary())

                            OLS Regression Results                            
Dep. Variable:     Retail_Short_Ratio   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     7.732
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           9.06e-06
Time:                        20:54:47   Log-Likelihood:                 580.53
No. Observations:                 184   AIC:                            -1151.
Df Residuals:                     179   BIC:                            -1135.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [36]:
model_scaled3 = sm.OLS(clean_data_scaled['Retail_Put_Share'], X1_scaled).fit()
print(model_scaled3.summary())

                            OLS Regression Results                            
Dep. Variable:       Retail_Put_Share   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                     5.172
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           0.000576
Time:                        20:55:34   Log-Likelihood:                 311.03
No. Observations:                 184   AIC:                            -612.1
Df Residuals:                     179   BIC:                            -596.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [44]:
# Retail_Short_Ratio

model_scaled6 = sm.OLS(clean_data_scaled['Retail_Short_Ratio'], X1_scaled).fit()
print(model_scaled6.summary())

                            OLS Regression Results                            
Dep. Variable:     Retail_Short_Ratio   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     7.732
Date:                Wed, 12 Nov 2025   Prob (F-statistic):           9.06e-06
Time:                        21:03:48   Log-Likelihood:                 580.53
No. Observations:                 184   AIC:                            -1151.
Df Residuals:                     179   BIC:                            -1135.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [40]:
# Adjusting calculations based on Bank Nifty Index specifc data
clean_data_scaled['Bank_Nifty_Total_OI'] = (
    data['Total Call Options - Open Interest BANK NIFTY INDEX'] + 
    data['Total Put Options - Open Interest BANK NIFTY INDEX'] + 
    data['Total Futures - Open Interest BANK NIFTY INDEX'])

# Y1: Retail Share
clean_data_scaled['Retail_Share_Bank_NIFTY'] = clean_data_scaled['Bank_Nifty_Total_OI'] * (data['Client_Total_OI'] / data['Total_OI'])

# Run regression with scaled variables
X2_scaled = clean_data_scaled[['DayOfWeek', 'NSE VIX - India', 
                                'Returns - BANK NIFTY INDEX']]
X2_scaled = sm.add_constant(X1_scaled)

model_scaled4 = sm.OLS(clean_data_scaled['Retail_Share_Bank_NIFTY'], X2_scaled).fit()
print(model_scaled4.summary())

                               OLS Regression Results                              
Dep. Variable:     Retail_Share_Bank_NIFTY   R-squared:                       0.133
Model:                                 OLS   Adj. R-squared:                  0.113
Method:                      Least Squares   F-statistic:                     6.850
Date:                     Wed, 12 Nov 2025   Prob (F-statistic):           3.75e-05
Time:                             20:57:30   Log-Likelihood:                -2644.5
No. Observations:                      184   AIC:                             5299.
Df Residuals:                          179   BIC:                             5315.
Df Model:                                4                                         
Covariance Type:                 nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [ ]:
# Retail Put Share for Bank Nifty Index

clean_data_scaled['Client_Bank_NIFTY_Put_Share'] = data['Total Put Options - Open Interest BANK NIFTY INDEX'] / (data['Total Put Options - Open Interest BANK NIFTY INDEX'] + data['Total Call Options - Open Interest BANK NIFTY INDEX']) *  (data['Client_Total_OI'] / data['Total_OI'])

model_scaled5 = sm.OLS(clean_data_scaled['Client_Bank_NIFTY_Put_Share'], X2_scaled).fit()
print(model_scaled5.summary())

                                 OLS Regression Results                                
Dep. Variable:     Client_Bank_NIFTY_Put_Share   R-squared:                       0.468
Model:                                     OLS   Adj. R-squared:                  0.456
Method:                          Least Squares   F-statistic:                     39.30
Date:                         Wed, 12 Nov 2025   Prob (F-statistic):           1.35e-23
Time:                                 21:01:54   Log-Likelihood:                 477.78
No. Observations:                          184   AIC:                            -945.6
Df Residuals:                              179   BIC:                            -929.5
Df Model:                                    4                                         
Covariance Type:                     nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------